In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import pickle
train = pickle.load(open('train.pkl', 'rb'))
test = pickle.load(open('test.pkl', 'rb'))
y = pickle.load(open('target.pkl', 'rb'))

test_ids = test.ID.values
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

SEED = 411

In [3]:
%%time
print("rounding...")
for i in train.columns:
    train[i] = np.round( np.log1p(train[i]) , decimals=4 )
    test[i] = np.round( np.log1p(test[i]) , decimals=4 )

gc.collect()
print(train.head(5))

rounding...
   48df886f9  0deb4b6a8  34b15f335  a8cb14b00  2f0771a37  30347e683  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   d08d1fbe3  6ee66e115  20aa07010  dc5a8f1d8    ...      3ecc09859  \
0        0.0        0.0     0.0000        0.0    ...            0.0   
1        0.0        0.0    14.6040        0.0    ...            0.0   
2        0.0        0.0     0.0000        0.0    ...            0.0   
3        0.0        0.0     0.0000        0.0    ...            0.0   
4        0.0        0.0    14.5087        0.0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0        0.0        0.0        0.0       

In [4]:
%%time
print("To String...")

tmp = train[train.columns[0]].apply(str)
tmp[ tmp=='0.0' ] = ''
CV = pd.DataFrame()
CV['features'] = tmp


tmp = test[test.columns[0]].apply(str)
tmp[ tmp=='0.0' ] = ''
CV_TE = pd.DataFrame()
CV_TE['features'] = tmp

for feat in train.columns[1:]:
    tmp = train[feat].apply(str)
    tmp[ tmp=='0.0' ] = ''
    CV['features'] = CV['features'] + tmp + ' '
    
    tmp = test[feat].apply(str)
    tmp[ tmp=='0.0' ] = ''
    CV_TE['features'] = CV_TE['features'] + tmp + ' '

del train, test
gc.collect()
print( CV )

To String...
                                               features
0                                      14.0779      ...
1            14.604                                 ...
2                                                   ...
3                                                   ...
4            14.5087                                ...
5            16.6499  8.9873       15.4249         9...
6                                                   ...
7                                                   ...
8            10.9682     9.9988                     ...
9                                                   ...
10                                    15.0393       ...
11                                     16.2314      ...
12                                                  ...
13                15.4249                  14.5087  ...
14                                                  ...
15                                                  ...
16           8.8538                

In [5]:
%%time
rd = CountVectorizer( lowercase=True, ngram_range=(1, 1), max_df=0.99, min_df=2)
train = rd.fit_transform( CV['features'] )
test = rd.transform(CV_TE['features'])
del rd, CV
gc.collect()
print(train.shape)

(4459, 5699)
Wall time: 13.7 s


In [6]:
old_train = pickle.load(open('train.pkl', 'rb'))
raw_cols = old_train.columns.tolist()
raw_cols.remove('ID')
del old_train; gc.collect()

0

In [7]:
feature_set_name = 'raw_simple_row_unsupervise_0_nan_class_features_ridge_lasso_oof_with_cat_encode'
train_feature_set = pickle.load(open('features/train_'+feature_set_name, 'rb'))
test_feature_set = pickle.load(open('features/test_'+feature_set_name, 'rb'))

In [8]:
train_feature_set.drop(raw_cols, axis=1, inplace=True)
test_feature_set.drop(raw_cols, axis=1, inplace=True)

In [9]:
train_feature_set.shape, test_feature_set.shape

((4459, 226), (49342, 226))

In [10]:
train = train.todense()
test = test.todense()

In [11]:
train = pd.DataFrame(data=train, columns=['count_vec_'+str(i) for i in range(train.shape[1])])
test = pd.DataFrame(data=test, columns=['count_vec_'+str(i) for i in range(train.shape[1])])

In [12]:
train = pd.concat([train, train_feature_set], axis=1)
test = pd.concat([test, test_feature_set], axis=1)

In [13]:
train

,count_vec_0,count_vec_1,count_vec_2,count_vec_3,count_vec_4,count_vec_5,count_vec_6,count_vec_7,count_vec_8,count_vec_9,...,class_6_6_lasso_binned_oof_mean,class_6_6_lasso_binned_oof_std,class_7_7_lasso_binned_oof_mean,class_7_7_lasso_binned_oof_std,class_8_8_lasso_binned_oof_mean,class_8_8_lasso_binned_oof_std,class_9_9_lasso_binned_oof_mean,class_9_9_lasso_binned_oof_std,class_10_10_lasso_binned_oof_mean,class_10_10_lasso_binned_oof_std
0,0,0,0,0,0,0,0,0,0,0,...,14.789060,0.287973,14.254282,0.184329,14.733241,0.317591,14.691398,0.293252,14.861382,0.392457
1,0,0,0,0,0,0,0,0,0,0,...,14.829526,0.245370,14.254282,0.184329,14.671664,0.207312,14.691398,0.293252,14.800125,0.193987
2,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.408127,0.214803,14.204297,0.090197,14.229414,0.134232
3,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.671664,0.207312,14.426236,0.207675,14.229414,0.134232
4,0,0,0,0,0,0,0,0,0,1,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
5,0,32,0,0,0,0,0,0,0,1,...,14.236154,0.151567,14.878262,0.234322,14.822069,0.251629,14.960813,0.332910,14.803191,0.271564
6,0,0,0,0,0,0,0,0,0,0,...,14.261041,0.113947,14.254558,0.112001,14.208796,0.095189,14.204297,0.090197,14.259061,0.112694
7,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.822069,0.251629,14.691398,0.293252,14.229414,0.134232
8,0,0,0,0,0,0,0,0,0,0,...,14.789060,0.287973,14.254282,0.184329,14.733241,0.317591,14.691398,0.293252,14.998782,0.246918
9,0,0,0,0,0,0,0,0,0,2,...,14.789060,0.287973,14.254282,0.184329,14.671664,0.207312,14.691398,0.293252,14.803191,0.271564


In [14]:
test

,count_vec_0,count_vec_1,count_vec_2,count_vec_3,count_vec_4,count_vec_5,count_vec_6,count_vec_7,count_vec_8,count_vec_9,...,class_6_6_lasso_binned_oof_mean,class_6_6_lasso_binned_oof_std,class_7_7_lasso_binned_oof_mean,class_7_7_lasso_binned_oof_std,class_8_8_lasso_binned_oof_mean,class_8_8_lasso_binned_oof_std,class_9_9_lasso_binned_oof_mean,class_9_9_lasso_binned_oof_std,class_10_10_lasso_binned_oof_mean,class_10_10_lasso_binned_oof_std
0,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
1,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
2,0,0,0,0,0,0,0,0,0,0,...,14.261041,0.113947,14.254558,0.112001,14.208796,0.095189,14.204297,0.090197,14.259061,0.112694
3,0,0,0,0,0,0,0,0,0,0,...,14.876033,0.384743,14.881701,0.398609,14.864462,0.396218,14.869545,0.397659,14.861382,0.392457
4,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
5,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
6,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
7,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
8,0,0,0,0,0,0,0,0,0,0,...,14.236154,0.151567,14.254282,0.184329,14.208796,0.095189,14.204297,0.090197,14.229414,0.134232
9,0,0,0,0,0,0,0,0,0,0,...,14.901044,0.260430,14.254282,0.184329,14.671664,0.207312,14.691398,0.293252,14.229414,0.134232


In [15]:
pickle.dump(train, open('features/train_count_vec', 'wb'))
pickle.dump(test, open('features/test_count_vec', 'wb'))